In [ ]:
# Dependency
import os
import copy
import pandas as pd
import numpy as np
from PIL import Image

import torch
from torch import nn

import torchvision.models as models
from torchvision import transforms
from sklearn.metrics import r2_score

from efficientnet_pytorch import EfficientNet

import time
import random

prefix = '/home/wx/work/particle_formation/' #remember to change it!
TRAIN_PATH = prefix + 'data/train/'
TEST_PATH = prefix + 'data/test/'
train_template = '>>> {} === Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
test_template = '\n>>> {} === Test Accuracy of the model on the {} test samples: {} %\n'
datamean = 4.924873
datastd = 1.877537
weight = [0.1631, 0.1557, 0.1598, 0.1731, 0.1708, 0.1776] # PCA-1st
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# class dataset
class DispersionDataSet(torch.utils.data.Dataset):
    def __init__(self, file_path, exp_index, cat=False, combined=False, transform=transforms.ToTensor()):
        tmp_df = pd.read_csv(file_path, converters={'image':str})
        self.transform = transform
        self.data_path = tmp_df['image']
        self.exp_index = exp_index
        self.cat = cat
        self.combined = combined
        if exp_index == 0:
            if not self.combined:
                self.labels = tmp_df['total_rating'].astype(np.float32)
            else:
                self.labels = tmp_df['total_rating'].astype(np.float32)
                self.ypos = tmp_df.loc[:,['r1','r2','r3','r4']].astype(np.float32)
        else:
            self.labels = tmp_df['exp' + str(exp_index)].astype(np.int32) - 1
    
    def __getitem__(self, index):
        image = Image.open(TRAIN_PATH + str(self.data_path[index]) + '.bmp' )
        if self.cat:
            img1 = image.crop((40, 40, 350, 220))
            img2 = image.crop((40, 320, 350, 440))
            img3 = image.crop((40, 440, 350, 660))
            if self.transform is not None:
                img1 = self.transform(img1)
                img2 = self.transform(img2)
                img3 = self.transform(img3)
            
            imgs = torch.cat([img1, img2, img3], 0)
            poss = torch.from_numpy(np.array(self.ypos.iloc[index]))
            if self.exp_index == 0:
                if self.combined:
                    x = torch.cat((imgs.flatten(0), poss.flatten(0)), -1)
                    label = torch.from_numpy(np.array(self.labels[index]))
                else:
                    x = torch.cat([img1, img2, img3], 0)
                    label = torch.from_numpy(np.array(self.labels[index]))
            else:
                x = torch.cat([img1, img2, img3], 0)
                label = torch.from_numpy(np.array(self.labels[index])).long()
            return x, label    
        else:
            if self.transform is not None:
                x = self.transform(image)
            
            imgs = self.transform(image)
            poss = torch.from_numpy(np.array(self.ypos.iloc[index]))
            if self.exp_index == 0:
                if self.combined:
                    x = torch.cat((imgs.flatten(0), poss.flatten(0)), -1)
                    label = torch.from_numpy(np.array(self.labels[index]))
                else:
                    x = self.transform(image)
                    label = torch.from_numpy(np.array(self.labels[index]))
            else:
                x = self.transform(image)
                label = torch.from_numpy(np.array(self.labels[index])).long()
            return x, label
        
    def __len__(self):
        return len(self.data_path)

class PredictDataSet(torch.utils.data.Dataset):
    def __init__(self, file_path, exp_index, cat=False, combined=False, transform=transforms.ToTensor()):
        tmp_df = pd.read_csv(file_path, converters={'image':str})
        self.transform = transform
        self.data_path = tmp_df['image']
        self.exp_index = exp_index
        self.cat = cat
        self.combined = combined
        if self.combined:
            self.ypos = tmp_df.loc[:,['r1','r2','r3','r4']].astype(np.float32)
    
    def __getitem__(self, index):
        image = Image.open(TRAIN_PATH + str(self.data_path[index]) + '.bmp' )
        if self.cat:
            img1 = image.crop((40, 40, 350, 220))
            img2 = image.crop((40, 320, 350, 440))
            img3 = image.crop((40, 440, 350, 660))
            if self.transform is not None:
                img1 = self.transform(img1)
                img2 = self.transform(img2)
                img3 = self.transform(img3)

            imgs = torch.cat([img1, img2, img3], 0)
            poss = torch.from_numpy(np.array(self.ypos.iloc[index]))
            if self.exp_index == 0 and self.combined:
                x = torch.cat((imgs.flatten(0), poss.flatten(0)), -1)
            else:
                x = torch.cat([img1, img2, img3], 0)
            return x   
        else:
            if self.transform is not None:
                x = self.transform(image)
            imgs = self.transform(image)
            poss = torch.from_numpy(np.array(self.ypos.iloc[index]))
            if self.exp_index == 0 and self.combined:
                x = torch.cat((imgs.flatten(0), poss.flatten(0)), -1)
            else:
                x = self.transform(image)
            return x
        
    def __len__(self):
        return len(self.data_path)      

In [ ]:
# networks
class navienet(nn.Module):
    def __init__(self, num_classes=9):
        super(navienet, self).__init__()
        self.main_layer = nn.Sequential(
            nn.Linear(224 * 224, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.main_layer(x.view(x.size(0), -1))

class navie_regnet(nn.Module):
    def __init__(self):
        super(navie_regnet, self).__init__()
        self.main_layer = nn.Sequential(
            nn.Linear(224 * 224, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        return self.main_layer(x.view(x.size(0), -1)).squeeze() * 8

class stayup_net(nn.Module):
    def __init__(self, res):
        super(stayup_net, self).__init__()
        self.model = res
        self.to(device)

    def forward(self, x):
        return self.model(x)
    
class pc_regressor(nn.Module):
    def __init__(self):
        super(pc_regressor, self).__init__()
        self.main_layer = nn.Sequential(
            nn.Sigmoid()
        )
        self.fc = nn.Linear(4, 1)
        self.to(device)

    def forward(self, x):
        out = self.main_layer(self.fc(x)) * 8.0 + 1
        return out

class exp_wrapper(nn.Module):
    def __init__(self, m0, m1, m2, m3, m4, m5):
        super(exp_wrapper, self).__init__()
        self.modules = [m0, m1, m2, m3, m4, m5]
        self.to(device)
        
    def forward(self, x):
        o0 = self.modules[0](x)
        o1 = self.modules[1](x)
        o2 = self.modules[2](x)
        o3 = self.modules[3](x)
        o4 = self.modules[4](x)
        o5 = self.modules[5](x)
        return torch.mean(torch.argmax(torch.stack([o0, o1, o2, o3, o4, o5]), dim=-1).float() + 1.0, dim=0, keepdim=True)

class combined_reg(nn.Module):
    def __init__(self, num_classes=9, plain_res=False):
        super(combined_reg, self).__init__()
        self.num_classes = num_classes
        self.plain_res = plain_res
        self.basics = []
        for i in range(6):
            if plain_res:
                res = models.resnet18(pretrained=True)
                res.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
                res.relu = nn.Sigmoid()
                # res.fc = nn.Linear(512, self.num_classes)
                self.basics.append(res)
            else:
                res = models.resnet18(pretrained=True)
                res.relu = nn.Sigmoid()
                res.fc = nn.Linear(512, self.num_classes)
                self.basics.append(stayup_net(res))
                self.basics[i].load_state_dict(torch.load(prefix+'model/stayup-exp' + str(i + 1) + '-1.ckpt'))

        self.exp = exp_wrapper(self.basics[0], self.basics[1], self.basics[2], 
                               self.basics[3], self.basics[4], self.basics[5])
        self.reg = pc_regressor()
        self.main_layer = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )
    # x: tensor
    # size [num_epoch, (3 or 1) * 224 * 224 + 4]
    # - x0: Tensor size[num_epoch, 3, 224, 224] or Tensor size[num_epoch, 1, 224, 224]
    # - x1: Tensor size[num_epoch, 4]
    def forward(self, x):
        x0 = x[:, 0 : x.shape[-1] - 4]
        x1 = x[:, x.shape[-1] - 4 : x.shape[-1]]
        x0 = x0.view(x0.shape[0], -1, 224, 224).to(device)
        x1 = x1.view(x1.shape[0], 4).to(device)
        # print(self.exp(x0).shape)
        # print(self.reg(x1).shape)
        out = self.main_layer(torch.cat((torch.t(self.exp(x0)), self.reg(x1)), dim=-1)) * 8.0 + 1
        return out
        

In [ ]:
# good stuff
class ModelManager:
    def __init__(self, model_name, model_path, train_path, test_path='', predict_path='', exp_index=0):
        self.exp_index = exp_index
        self.model_name = model_name
        self.model_path = model_path
        self.train_path = train_path
        self.test_path = test_path
        self.predict_path = predict_path

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        self.num_epochs = 32
        self.learning_rate = 0.0005
        self.batch_size = 16
        if not self.exp_index == 0:
            self.num_classes = 9
        else:
            self.num_classes = 1
        self.print_len = 2
        self.wd = 0

        if not self.exp_index == 0:
            self.criterion = nn.CrossEntropyLoss()
        else:
            self.criterion = nn.MSELoss()
        self.transformer = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        #    transforms.Normalize(mean = (0.5,), std = (0.5,))
        ])
        self.get_model(init=True)
        self.optimizer = torch.optim.Adamax(
            params=self.model.parameters(),
            lr=self.learning_rate,
            weight_decay=self.wd
        )
    
    def _get_data_loader(self, path, batch_size, predict=False, shuffle=True, cat=False, combined=False):
        if not predict:
            dataset = DispersionDataSet(path, self.exp_index, cat, combined, self.transformer)
        else:
            dataset = PredictDataSet(path, cat, combined, self.transformer)
        return torch.utils.data.DataLoader(
                    dataset=dataset,
                    batch_size=batch_size,
                    num_workers=4,
                    shuffle=shuffle
                )
          
    def get_model(self, init=False, train_again=False):
        if init:
            if 'resnet18' in self.model_name:
                self.model = models.resnet18(pretrained=True)
                self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
                self.model.relu = nn.Sigmoid()
            #    self.model.fc = nn.Linear(512, self.num_classes)
            elif 'resnet50' in self.model_name:
                self.model = models.resnet50(pretrained=True)
                self.model.relu = nn.Sigmoid()
                self.model.fc = nn.Linear(2048, self.num_classes)  
            elif 'efficient' in self.model_name:
                self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=self.num_classes)
            elif 'navienet' in self.model_name:
                self.model = navienet(self.num_classes)
            elif 'navie_regnet' in self.model_name:
                self.model = navie_regnet()
            elif 'stayup' in self.model_name:
                res = models.resnet18(pretrained=True)
                #res.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
                res.relu = nn.Sigmoid()
                res.fc = nn.Linear(512, self.num_classes)
                self.model = stayup_net(res.cuda())
            elif 'combined_reg' in self.model_name:
                self.model = combined_reg(plain_res=False)
            else:
                None
        else:
            if train_again or not os.path.exists(self.model_path):
                if 'stayup' in self.model_name:
                    data_loader = self._get_data_loader(self.train_path, batch_size=self.batch_size, shuffle=True, cat=True)
                elif 'combined_reg' in self.model_name and not self.model.plain_res:
                    data_loader = self._get_data_loader(self.train_path, batch_size=self.batch_size, shuffle=True, cat=True, combined=True)
                else:
                    data_loader = self._get_data_loader(self.train_path, batch_size=self.batch_size, shuffle=True)
                self.train_it(data_loader, self.print_len, self.model_name)
            else:
                self.model.load_state_dict(torch.load(self.model_path))

            if(train_again or not os.path.exists(self.model_path)):
                torch.save(self.model.state_dict(), self.model_path)
    
    def train_it(self, train_data, print_len, log = ''):
        self.model = self.model.to(self.device)
        total_steps = len(train_data)
        loss_list = []
        for ep in range(self.num_epochs):
            for i, (x, y) in enumerate(train_data):
                if self.device.type == 'cuda':
                    torch.cuda.empty_cache()

                x = x.to(self.device)
                y = y.to(self.device)
                if 'Inception' in log:
                    out, aux = self.model(x)
                    loss1 = self.criterion(out, y)
                    loss2 = self.criterion(aux, y)
                    loss = loss1 + 0.4*loss2
                else:
                    out = self.model(x).squeeze()
                    loss = self.criterion(out, y)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                loss_list.append(loss.item())
                if (i + 1) % print_len == 0:
                    print(
                        train_template.format(
                            log, ep+1, self.num_epochs, i+1, total_steps, loss.item()
                        ))
        return loss_list

    def test_it(self, log = ''):
        self.model = self.model.to(self.device)
        self.model.eval()
        if 'combined_reg' in self.model_name:
            test_data = self._get_data_loader(self.test_path, batch_size=1, shuffle=False, cat=True, combined=True)
        elif 'stayup' in self.model_name:
            test_data = self._get_data_loader(self.test_path, batch_size=1, shuffle=False, cat=True)
        else:
            test_data = self._get_data_loader(self.test_path, batch_size=1, shuffle=False)
        predict_list = []
        label_list = []
        with torch.no_grad():
            for x, y in test_data:
                if self.device.type == 'cuda':
                    torch.cuda.empty_cache()
                x = x.to(self.device)
                y = y.to(self.device)

                out = self.model(x)
                
                if not self.exp_index == 0:
                    pre = torch.argmax(out)
                    predict_list.append(pre.cpu().item())
                else:
                    predict_list.append(out.data.cpu().item())
                label_list.append(y.cpu().item())

        print(test_template.format(log, len(predict_list), 100 * r2_score(label_list, predict_list)))
        print('precision: {}'.format(np.sum(np.asarray(label_list) == np.asarray(predict_list)) / len(predict_list)))
        print('label: {}'.format(label_list))
        print('predict: {}'.format(predict_list))
        print('r2dist: {}'.format(np.square(np.asarray(label_list) - np.asarray(predict_list))))

    def predict_it(self):
        self.model = self.model.to(self.device)
        self.model.eval()
        if 'combined_reg' in self.model_name:
            predict_data = self._get_data_loader(self.test_path, batch_size=1, predict=True, shuffle=False, cat=True, combined=True)
        elif 'stayup' in self.model_name:
            predict_data = self._get_data_loader(self.test_path, batch_size=1, predict=True, shuffle=False, cat=True)
        else:
            predict_data = self._get_data_loader(self.test_path, batch_size=1, predict=True, shuffle=False)
        predict_list = []
        with torch.no_grad():
            for x in predict_data:
                if self.device.type == 'cuda':
                    torch.cuda.empty_cache()
                
                x = x.to(self.device)
                out = self.model(x)
                if not self.exp_index == 0:
                    pre = torch.argmax(out.data) + 1
                    predict_list.append(pre.cpu().item())
                else:
                    predict_list.append(out.data.cpu().item())
        
        return predict_list

In [ ]:
# for offline testing
def get_mydataset(train_name, test_name, train_ratio=0.7, path=prefix + 'data/train_scores.csv', all=True):
    train_df = pd.read_csv(path)
    train_data_path = train_df['image']
    if all:
        train_labels = np.asarray([train_df['exp1'], train_df['exp2'], train_df['exp3'], train_df['exp4'], train_df['exp5'], train_df['exp6'], train_df['total_rating']])
    else:
        train_labels = np.asarray([train_df['total_rating']])
    
    mytrain_num = int(train_ratio * 197)
    mytest_num = 197 - mytrain_num
    if all:
        mytrain_dict = {'image':[], 'exp1':[], 'exp2':[], 'exp3':[], 'exp4':[], 'exp5':[], 'exp6':[], 'total_rating':[]}
        mytest_dict = {'image':[], 'exp1':[], 'exp2':[], 'exp3':[], 'exp4':[], 'exp5':[], 'exp6':[], 'total_rating':[]}
    else:
        mytrain_dict = {'image':[], 'total_rating':[]}
        mytest_dict = {'image':[], 'total_rating':[]}
    index = [x for x in range(197)]
    random.shuffle(index)

    for it in range(mytrain_num):
        mytrain_dict['image'].append(train_data_path[it])
        if all:
            mytrain_dict['exp1'].append(train_labels[0][it])
            mytrain_dict['exp2'].append(train_labels[1][it])
            mytrain_dict['exp3'].append(train_labels[2][it])
            mytrain_dict['exp4'].append(train_labels[3][it])
            mytrain_dict['exp5'].append(train_labels[4][it])
            mytrain_dict['exp6'].append(train_labels[5][it])
            mytrain_dict['total_rating'].append(train_labels[6][it])
        else:
            mytrain_dict['total_rating'].append(train_labels[0][it])

    for it in range(mytrain_num, 197):
        mytest_dict['image'].append(train_data_path[it])
        if all:
            mytest_dict['exp1'].append(train_labels[0][it])
            mytest_dict['exp2'].append(train_labels[1][it])
            mytest_dict['exp3'].append(train_labels[2][it])
            mytest_dict['exp4'].append(train_labels[3][it])
            mytest_dict['exp5'].append(train_labels[4][it])
            mytest_dict['exp6'].append(train_labels[5][it])
            mytest_dict['total_rating'].append(train_labels[6][it])
        else:
            mytest_dict['total_rating'].append(train_labels[0][it])

    pd.DataFrame(data=mytrain_dict).to_csv(prefix + 'data/' + train_name, index=False)
    pd.DataFrame(data=mytest_dict).to_csv(prefix + 'data/' + test_name, index=False)

In [ ]:
# get_mydataset('mytrain_pca.csv', 'mytest_pca.csv', train_ratio=0.7, path=prefix + 'data/train_score_pca.csv', all=False)

In [ ]:
exp1_resnet18 = ModelManager(model_name='resnet18-exp1',
                             model_path=prefix+'model/resnet18-exp1-814.ckpt',
                             train_path=prefix+'data/train_scores.csv',
                             test_path=prefix+'data/train_scores.csv', 
                             predict_path=prefix+'data/to_predict.csv', exp_index=1)
exp1_resnet18.get_model(train_again=False)
exp1_resnet18.test_it(log='test on test: ')
pred_list_exp1_resnet18 = exp1_resnet18.predict_it()

In [ ]:
exp2_resnet18 = ModelManager(model_name='resnet18-exp2',
                             model_path=prefix+'model/resnet18-exp2-814.ckpt',
                             train_path=prefix+'data/train_scores.csv',
                             test_path=prefix+'data/train_scores.csv',
                             predict_path=prefix+'data/to_predict.csv', exp_index=2)
exp2_resnet18.get_model(train_again=False)
exp2_resnet18.test_it(log='test on test: ')
pred_list_exp2_resnet18 = exp2_resnet18.predict_it()

In [ ]:
exp3_resnet18 = ModelManager(model_name='resnet18-exp3',
                             model_path=prefix+'model/resnet18-exp3-814.ckpt',
                             train_path=prefix+'data/train_scores.csv',
                             test_path=prefix+'data/train_scores.csv',
                             predict_path=prefix+'data/to_predict.csv', exp_index=3)
exp3_resnet18.get_model(train_again=False)
exp3_resnet18.test_it(log='test on test: ')
pred_list_exp3_resnet18 = exp3_resnet18.predict_it()

In [ ]:
exp4_resnet18 = ModelManager(model_name='resnet18-exp4',
                             model_path=prefix+'model/resnet18-exp4-814.ckpt',
                             train_path=prefix+'data/train_scores.csv',
                             test_path=prefix+'data/train_scores.csv',
                             predict_path=prefix+'data/to_predict.csv', exp_index=4)
exp4_resnet18.get_model(train_again=False)
exp4_resnet18.test_it(log='test on test: ')
pred_list_exp4_resnet18 = exp4_resnet18.predict_it()

In [ ]:
exp5_resnet18 = ModelManager(model_name='resnet18-exp5',
                             model_path=prefix+'model/resnet18-exp5-814.ckpt',
                             train_path=prefix+'data/train_scores.csv',
                             test_path=prefix+'data/train_scores.csv',
                             predict_path=prefix+'data/to_predict.csv', exp_index=5)
exp5_resnet18.get_model(train_again=False)
exp5_resnet18.test_it(log='test on test: ')
pred_list_exp5_resnet18 = exp5_resnet18.predict_it()

In [ ]:
exp6_resnet18 = ModelManager(model_name='resnet18-exp6',
                             model_path=prefix+'model/resnet18-exp6-814.ckpt',
                             train_path=prefix+'data/train_scores.csv',
                             test_path=prefix+'data/train_scores.csv',
                             predict_path=prefix+'data/to_predict.csv', exp_index=6)
exp6_resnet18.get_model(train_again=False)
exp6_resnet18.test_it(log='test on test: ')
pred_list_exp6_resnet18 = exp6_resnet18.predict_it()

In [ ]:
exp1_stayup = ModelManager(model_name='stayup-exp1',
                          model_path=prefix+'model/stayup-exp1-1.ckpt',
                          train_path=prefix+'data/train_scores_bigger.csv',
                          test_path=prefix+'data/train_scores_bigger.csv',
                          predict_path=prefix+'data/to_predict.csv', exp_index=1)
exp1_stayup.get_model(train_again=False)
exp1_stayup.test_it(log='test on test: ')
pred_list_exp1_stayup = exp1_stayup.predict_it()

exp2_stayup = ModelManager(model_name='stayup-exp2',
                          model_path=prefix+'model/stayup-exp2-1.ckpt',
                          train_path=prefix+'data/train_scores_bigger.csv',
                          test_path=prefix+'data/train_scores_bigger.csv',
                          predict_path=prefix+'data/to_predict.csv', exp_index=2)
exp2_stayup.get_model(train_again=False)
exp2_stayup.test_it(log='test on test: ')
pred_list_exp2_stayup = exp2_stayup.predict_it()

exp3_stayup = ModelManager(model_name='stayup-exp3',
                          model_path=prefix+'model/stayup-exp3-1.ckpt',
                          train_path=prefix+'data/train_scores_bigger.csv',
                          test_path=prefix+'data/train_scores_bigger.csv',
                          predict_path=prefix+'data/to_predict.csv', exp_index=3)
exp3_stayup.get_model(train_again=False)
exp3_stayup.test_it(log='test on test: ')
pred_list_exp3_stayup = exp3_stayup.predict_it()

exp4_stayup = ModelManager(model_name='stayup-exp4',
                          model_path=prefix+'model/stayup-exp4-1.ckpt',
                          train_path=prefix+'data/train_scores_bigger.csv',
                          test_path=prefix+'data/train_scores_bigger.csv',
                          predict_path=prefix+'data/to_predict.csv', exp_index=4)
exp4_stayup.get_model(train_again=False)
exp4_stayup.test_it(log='test on test: ')
pred_list_exp4_stayup = exp4_stayup.predict_it()

exp5_stayup = ModelManager(model_name='stayup-exp5',
                          model_path=prefix+'model/stayup-exp5-1.ckpt',
                          train_path=prefix+'data/train_scores_bigger.csv',
                          test_path=prefix+'data/train_scores_bigger.csv',
                          predict_path=prefix+'data/to_predict.csv', exp_index=5)
exp5_stayup.get_model(train_again=False)
exp5_stayup.test_it(log='test on test: ')
pred_list_exp5_stayup = exp5_stayup.predict_it()

exp6_stayup = ModelManager(model_name='stayup-exp6',
                          model_path=prefix+'model/stayup-exp6-1.ckpt',
                          train_path=prefix+'data/train_scores_bigger.csv',
                          test_path=prefix+'data/train_scores_bigger.csv',
                          predict_path=prefix+'data/to_predict.csv', exp_index=6)
exp6_stayup.get_model(train_again=False)
exp6_stayup.test_it(log='test on test: ')
pred_list_exp6_stayup = exp6_stayup.predict_it()

In [ ]:
# write ans to *.csv
def ans2csv(pred1, pred2, pred3, pred4, pred5, pred6, pred_path, result_path, all=False, weigh=False):
    if not weigh:
        pred_avg = (np.asarray(pred1) + np.asarray(pred2) + np.asarray(pred3) 
                 + np.asarray(pred4) + np.asarray(pred5) + np.asarray(pred6)) / 6.0
    else:
        pred_avg = (np.asarray(pred1)*weight[0] + np.asarray(pred2)*weight[1] + np.asarray(pred3)*weight[2] 
                 + np.asarray(pred4)*weight[3] + np.asarray(pred5)*weight[4] + np.asarray(pred6)*weight[5])
    if not all:
        pred_dict = {'image':[], 'total_rating':[]}

        for it, id in enumerate(pd.read_csv(pred_path)['image']):
            pred_dict['image'].append(id)
            pred_dict['total_rating'].append(round(pred_avg[it], 1))

        pd.DataFrame(data=pred_dict).to_csv(result_path, index=False)
    else:
        pred_dict = {'image':[], 'exp1':[], 'exp2':[], 'exp3':[], 
                    'exp4':[], 'exp5':[], 'exp6':[], 'total_rating':[]}
        for it, id in enumerate(pd.read_csv(pred_path)['image']):
            pred_dict['image'].append(id)
            pred_dict['exp1'].append(round(pred1[it], 1))
            pred_dict['exp2'].append(round(pred2[it], 1))
            pred_dict['exp3'].append(round(pred3[it], 1))
            pred_dict['exp4'].append(round(pred4[it], 1))
            pred_dict['exp5'].append(round(pred5[it], 1))
            pred_dict['exp6'].append(round(pred6[it], 1))
            pred_dict['total_rating'].append(round(pred_avg[it], 1))

        pd.DataFrame(data=pred_dict).to_csv(result_path, index=False)

In [ ]:
'''
ans2csv(pred_list_exp1_resnet18, pred_list_exp2_resnet18, pred_list_exp3_resnet18,
        pred_list_exp4_resnet18, pred_list_exp5_resnet18, pred_list_exp6_resnet18,
        pred_path=prefix+'data/to_predict.csv',
        result_path=prefix + 'result/team6submission_resnet18_bs16_lr0.0005_withoutnorm_32epochs_sigmoid_ncori_avg.csv',
        all=False, weigh=False)
'''
ans2csv(pred_list_exp1_stayup, pred_list_exp2_stayup, pred_list_exp3_stayup,
        pred_list_exp4_stayup, pred_list_exp5_stayup, pred_list_exp6_stayup,
        pred_path=prefix+'data/to_predict.csv',
        result_path=prefix + 'result/team6submission_stayup_bs16_lr0.0005_withoutnorm_32epochs_sigmoid_nc9_avg_B.csv',
        all=False, weigh=False)
'''
ans2csv(pred_list_exp1_resnet18, pred_list_exp2_resnet18, pred_list_exp3_resnet18,
        pred_list_exp4_resnet18, pred_list_exp5_resnet18, pred_list_exp6_resnet18,
        pred_path=prefix+'data/to_predict.csv',
        result_path=prefix + 'result/predict_scores_resnet18_bs16_lr0.0005_withoutnorm_64epochs_nc9.csv',
        all=True)
'''
